### **Module1. Environment Initialize**

**Dataset URL**: 
**    **
https://www.kaggle.com/geomack/spotifyclassification/home#data.csv. 

The dataset used is the dataset of the 2017 song.Each row represents a song. There are 16 columns. 13 of which are song attributes, one column for song name, one for artist, and a column called "target" which is the label for the song.The attributes of the song are:
**     **
  *acousticness, danceability, durationms, energy, instrumentalness, key, liveness, loudness, mode, speechiness, tempo, timesignature, valence.*
**     **
Information on what those traits mean can be found here: 


https://developer.spotify.com/web-api/get-audio-features/

**    **

In [0]:
#Introducing Google Drive to store data sets
from google.colab import drive 

drive.mount('/content/gdrive/') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
#Install kaggle package
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"durkys12","key":"82e41a31888df03587b69009959159f8"}'}

In [0]:
!mkdir -p ~/.kaggle 

!cp kaggle.json ~/.kaggle/ 

!chmod 600 ~/.kaggle/kaggle.json 

In [0]:
!kaggle datasets download -d geomack/spotifyclassification -p '../content/gdrive/My Drive'


spotifyclassification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
!unzip -q -n '../content/gdrive/My Drive/spotifyclassification.zip' -d '../content/gdrive/My Drive/'
#!7z e '../content/gdrive/My Drive/Iceberg Classifier Challenge/train.json.7z' -o'../content/gdrive/My Drive/Iceberg Classifier Challenge'

In [0]:
import pandas as pd
df = pd.read_csv('../content/gdrive/My Drive//data.csv')
data = df
data

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,2012,0.00106,0.584,274404,0.932,0.002690,1,0.1290,-3.501,1,0.3330,74.976,4.0,0.211,0,Like A Bitch - Kill The Noise Remix,Kill The Noise
2013,2013,0.08770,0.894,182182,0.892,0.001670,1,0.0528,-2.663,1,0.1310,110.041,4.0,0.867,0,Candy,Dillon Francis
2014,2014,0.00857,0.637,207200,0.935,0.003990,0,0.2140,-2.467,1,0.1070,150.082,4.0,0.470,0,Habit - Dack Janiels & Wenzday Remix,Rain Man
2015,2015,0.00164,0.557,185600,0.992,0.677000,1,0.0913,-2.735,1,0.1330,150.011,4.0,0.623,0,First Contact,Twin Moons


### **Module2.Classification and input output**

**      **
The user can enter the name of one of the 2017 songs, the author, and make a query. If the input is incorrect, the query result will appear: "No such song". At the same time, the machine learning classification method is used to classify the song For different types (like or dislike), and show the results of preferences. (Tasks s1a and s1f)
**  **

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
train,test = train_test_split(data,test_size = 0.1, random_state = 2)
train.shape

(1815, 17)

In [0]:
train = train.drop(['Unnamed: 0','song_title','artist'], axis = 1)
test = test.drop(['Unnamed: 0','song_title','artist'], axis = 1)
y_train = pd.DataFrame(train['target']) 
X_train = train.drop(['target'],axis = 1)
y_test = pd.DataFrame(test['target']) 
X_test = test.drop(['target'],axis = 1)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

forest = RandomForestClassifier(n_estimators=250)
forest.fit(X_train,y_train.values.ravel())
y_pred_forest = forest.predict(X_test)

accuracyforest = accuracy_score(y_test,y_pred_forest)
accuracyforest

0.7722772277227723

In [0]:
while(True):
    song_title = input('please input song_title: ')
    artist = input('please input artist: ')    
    # song_title = 'Mask Off'
    # artist = 'Future'
    inputData = data[(data['song_title']==song_title)&(data['artist']==artist)]
    if(len(inputData)==0):
        print('no song found!')
        continue
    inputDataWithId = inputData.drop(['song_title','artist', 'target'], axis = 1)
    inputDataFromat = inputData.drop(['Unnamed: 0','song_title','artist', 'target'], axis = 1)
    for i in range(len(inputDataFromat)):
        pred = forest.predict(inputDataFromat.iloc[i:i+1,:])
        print('------ {} ------'.format(i))
        print(inputDataFromat.iloc[i:i+1,:])
        print('predict: {} true: {}'.format(pred[0], data.at[i, 'target']))
    break

please input song_title: Mask Off
please input artist: Future
------ 0 ------
   acousticness  danceability  duration_ms  ...    tempo  time_signature  valence
0        0.0102         0.833       204600  ...  150.062             4.0    0.286

[1 rows x 13 columns]
predict: 1 true: 1
------ 1 ------
      acousticness  danceability  duration_ms  ...    tempo  time_signature  valence
1173        0.0102         0.833       204600  ...  150.062             4.0    0.286

[1 rows x 13 columns]
predict: 1 true: 1


### **Module3.Control Vocabulary Function**

** **
The control vocabulary is established according to different attributes, and the first five results can also be output when the control word query is required. (Task s1c) Such as:

1. *The value of instrumentalness determines the audio content, "talkshow" or "poem" greater than 0.66,"hippop" less than 0.66 and greater than 0.33, and "Pure music" below 0.33.*

2. *Tempo is the speed, in units of heartbeats per minute (BPM), higher than 140 is "fast",100-140 is "middle", and below 100 is "slow".*

3. *The value of Valence determines the emotion of the song, above 0.5 corresponds to "happy", below 0.5 corresponds to "sad".*

4. *The value of dancebility is higher than 0.7 for "isDance", lower than 0.7 is "notDance"*

5. *According to duration_ms, "long song" is greater than 200,000 and "short song" is less than 200,000.*
**  **

In [0]:
def instrumentalnessHelper(x):
    if(x > 0.66):
        return 'talkshow'
    if(x > 0.33):
        return 'hippop'
    return 'Pure music'
def tempoHelper(x):
    if(x > 140):
        return 'fast'
    if(x > 100):
        return 'middle'
    return 'slow'
def ValenceHelper(x):
    if x > 0.5:
        return 'happy'
    return 'sad'
def dancebilityHelper(x):
    if(x > 0.7):
        return 'isDance'
    else:
        return 'notDance'
def durationMsHelper(x):
    if(x > 200000):
        return 'long song'
    return 'short song'
data['instrumentalnessName'] = data['instrumentalness'].apply(instrumentalnessHelper)
data['tempoName'] = data['tempo'].apply(tempoHelper)
data['valenceName'] = data['valence'].apply(ValenceHelper)
data['dancebilityName'] = data['danceability'].apply(dancebilityHelper)
data['duration_msName'] = data['duration_ms'].apply(durationMsHelper)
data

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,instrumentalnessName,tempoName,valenceName,dancebilityName,duration_msName
0,0,0.01020,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future,Pure music,fast,sad,isDance,long song
1,1,0.19900,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino,Pure music,fast,happy,isDance,long song
2,2,0.03440,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future,Pure music,slow,sad,isDance,short song
3,3,0.60400,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House,hippop,slow,sad,notDance,short song
4,4,0.18000,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys,hippop,fast,happy,notDance,long song
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,2012,0.00106,0.584,274404,0.932,0.002690,1,0.1290,-3.501,1,0.3330,74.976,4.0,0.211,0,Like A Bitch - Kill The Noise Remix,Kill The Noise,Pure music,slow,sad,notDance,long song
2013,2013,0.08770,0.894,182182,0.892,0.001670,1,0.0528,-2.663,1,0.1310,110.041,4.0,0.867,0,Candy,Dillon Francis,Pure music,middle,happy,isDance,short song
2014,2014,0.00857,0.637,207200,0.935,0.003990,0,0.2140,-2.467,1,0.1070,150.082,4.0,0.470,0,Habit - Dack Janiels & Wenzday Remix,Rain Man,Pure music,fast,sad,notDance,long song
2015,2015,0.00164,0.557,185600,0.992,0.677000,1,0.0913,-2.735,1,0.1330,150.011,4.0,0.623,0,First Contact,Twin Moons,talkshow,fast,happy,notDance,short song


In [0]:
print('instrumentalness: talkshow, hippop, Pure music')
print('tempo: fast, middle, slow')
print('valence: happy, sad')
print('dancebility: isDance, notDance')
print('duration_ms: long song, short song')
word = input('please input control word: ')
filterData = data[(data['instrumentalnessName'] == word)|
                 (data['tempoName'] == word)|
                 (data['valenceName'] == word)|
                 (data['dancebilityName'] == word)|
                 (data['duration_msName'] == word)]

filterData.sample(5)
#output 5 examples

instrumentalness: talkshow, hippop, Pure music
tempo: fast, middle, slow
valence: happy, sad
dancebility: isDance, notDance
duration_ms: long song, short song
please input control word: slow


,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,instrumentalnessName,tempoName,valenceName,dancebilityName,duration_msName
1989,1989,0.000797,0.480,222053,0.826,0.000001,0,0.1250,-4.602,1,0.0397,96.000,4.0,0.701,0,Different Colors,WALK THE MOON,Pure music,slow,happy,notDance,long song
567,567,0.164000,0.824,181800,0.513,0.000000,8,0.0487,-9.591,1,0.2080,99.938,4.0,0.568,1,Stomp - Dirty,5kinAndBone5,Pure music,slow,happy,isDance,short song
853,853,0.271000,0.567,336800,0.851,0.000000,1,0.5690,-5.897,1,0.4840,80.935,4.0,0.598,1,"A Day In The Life - feat. RZA, The Mars Volta ...",Handsome Boy Modeling School,Pure music,slow,happy,notDance,long song
1112,1112,0.078600,0.724,200813,0.904,0.000000,8,0.2260,-3.354,0,0.0966,90.999,4.0,0.851,0,DUELE EL CORAZON,Enrique Iglesias,Pure music,slow,happy,isDance,long song
1544,1544,0.846000,0.569,282320,0.117,0.852000,5,0.0978,-22.741,1,0.0446,60.483,4.0,0.102,0,When Sunny Gets Blue,McCoy Tyner,talkshow,slow,sad,notDance,long song


### **Module4.Song recommendation based on cosine similarity**

**   **
After obtaining the song name input by the user, the song is recommended according to the cosine similarity, and the top five with the highest similarity are output.
**    **

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings(action='ignore')

dataFormat = data.copy()
dataFormat = dataFormat[(dataFormat['song_title']!=song_title)&(dataFormat['artist']!=artist)]


dataFormat = dataFormat.drop(['target', 'instrumentalnessName', 'tempoName',
                'valenceName','dancebilityName','duration_msName'],axis=1)
dataFormatScaler = dataFormat.drop(['Unnamed: 0','song_title','artist'],axis=1)
dataFormat['similarity'] = 1.0
dataFormat['similarity'] = dataFormat['similarity'].astype(float)

mm = MinMaxScaler()
column = dataFormatScaler.columns.tolist()
dataFormatScaler[column]= mm.fit_transform(dataFormatScaler[column])
inputScaler = mm.transform(inputDataFromat.iloc[0:1,:])
similar = []
for i in range(len(dataFormat['similarity'])):
    sim = cosine_similarity(inputScaler, dataFormatScaler.iloc[i:i+1,:])
    similar.append(round(sim[0][0],3))
dataFormat['similarity'] = similar
dataFormat = dataFormat.sort_values(['similarity'], ascending=False)
recommend5 = dataFormat.head(5)

In [0]:
recommend5
#output 5 examples

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,song_title,artist,similarity
142,142,0.05120,0.851,224653,0.387,0.000000,0,0.0996,-13.584,1,0.334,116.629,4.0,0.365,Smell Yo D*ck,Riskay,0.985
267,267,0.09620,0.654,252095,0.292,0.003100,2,0.1060,-12.977,1,0.474,153.547,4.0,0.111,River,Ibeyi,0.984
148,148,0.09620,0.654,252095,0.292,0.003100,2,0.1060,-12.977,1,0.474,153.547,4.0,0.111,River,Ibeyi,0.984
1187,1187,0.06560,0.828,299333,0.433,0.000876,0,0.1220,-9.716,1,0.199,148.073,4.0,0.446,Lollipop,Lil Wayne,0.978
140,140,0.00469,0.721,228933,0.621,0.010300,1,0.1990,-4.378,1,0.306,159.967,4.0,0.578,Sittin' Sidewayz - feat. Big Pokey Explicit,Paul Wall,0.976


### **Module5.Schema.org Format Output**

**  **
*When the user selects a certain song, it should be output in two different schema.org formats (rdf and json). (Task s1e)*

**   **

In [0]:
index = int(input('please input index of the data(0-2016):'))
# id = 0
chooseData = data.iloc[[index],:]
chooseData

please input index of the data(0-2016):100


,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,instrumentalnessName,tempoName,valenceName,dancebilityName,duration_msName
100,100,0.228,0.63,315595,0.98,0.00661,1,0.487,-4.914,1,0.059,108.009,4.0,0.805,1,Slumlord,Neon Indian,Pure music,middle,happy,notDance,long song


In [0]:
chooseData

,Unnamed: 0,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist,instrumentalnessName,tempoName,valenceName,dancebilityName,duration_msName
100,100,0.228,0.63,315595,0.98,0.00661,1,0.487,-4.914,1,0.059,108.009,4.0,0.805,1,Slumlord,Neon Indian,Pure music,middle,happy,notDance,long song


In [0]:
chooseData.to_json('{}.json'.format(index))
print('export {}.json successfully'.format(index))

export 100.json successfully


In [0]:
!pip install rdflib

     |████████████████████████████████| 235kB 4.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 


In [0]:
import rdflib
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, SKOS, OWL, RDF, RDFS, VOID, XMLNS, XSD
chooseData= chooseData.rename(columns={'Unnamed: 0': 'id'})
columns = chooseData.columns.tolist()
g = rdflib.Graph()
for _,column in enumerate(columns):
    cc = column.replace(' ', '_')
    a = rdflib.URIRef("http://www.example.org/{}".format(index))
    b = rdflib.URIRef("http://www.example.org/{}".format(cc))
    if(type(chooseData.iloc[[0],[_]].values[0][0]) == str):
        dd = chooseData.iloc[[0],[_]].values[0][0].replace(' ', '_')
    else:
        dd = chooseData.iloc[[0],[_]].values[0][0]
    c = rdflib.URIRef("http://www.example.org/{}".format(dd))
    g.add((a,b,c))
g.serialize("{}.rdf".format(index))
print('export {}.rdf successfully'.format(index))

export 100.rdf successfully


In [0]:
data